In [ ]:
# Instalação da biblioteca para recuperação dos informações do UOL.

!pip install git+https://github.com/pyanderson/python-brasileirao

In [ ]:
# Instalação da API do GOOGLE BARD

!pip install bardapi

In [ ]:
# imports Python
import os
import json
import requests
import pandas as pd
from bardapi import Bard
from bardapi import BardCookies

## Fase Extração

#### Nesta fase recuperamos os dados do Brasileirão 2023 a partir da biblioteca disponível no endereço https://github.com/pyanderson/python-brasileirao  do site uol, em um dicionário.

In [ ]:
def get_equipes():
    
    URL = ('http://jsuol.com.br/c/monaco/utils/gestor/commons.js?callback='
           'simulador_dados_jsonp&file=commons.uol.com.br/sistemas/esporte'
           '/modalidades/futebol/campeonatos/dados/2023/30/dados.json')
    response = requests.get(URL)
    jn = json.loads(response.text[22:len(response.text)-3])
    return jn if response.status_code == 200 else None
    


## Tratamento
#### Montamos um Dataframe Panda a partir do dicionário carregado do UOL. O Dataframe contem os seguintes campos: 
#####              - Posicao                  : Posição da equipe no Campeonato Brasileiro
#####               - Clube                    : Nome da Equipe
#####               - Pontos Ganhos   : Pontos Ganhos até a rodada atual


In [ ]:
def transformacao(jn):
    equipes = {}
    classificacao = {'Posicao':[], "Clube":[],"Pontos Ganhos":[], 'Rodada Atual':[]}
    rodada = None
    
    nome_completo = jn['nome-completo']
    rodada = jn['fases']['3682']['rodada']
    for k, v in jn['equipes'].items():
        equipes[k] = v['nome-comum']

    fases = jn['fases']['3682']
    rodada = fases['rodada'] 
    for e, d in fases['classificacao']['equipe'].items():
        classificacao['Posicao'].append(d['pos'])
        classificacao['Clube'].append(equipes[e])
        classificacao['Pontos Ganhos'].append(d['pg']['total'])
        classificacao['Rodada Atual'].append(rodada['atual'])
    df = pd.DataFrame(classificacao)    
    return df
    

## Carregamento

#### Nesta fase iremos usar a Generative AI da Google (Bard) para tentarmos obter a probabilidade de um time ser campeão brasileiro com os pontos atuais,

### É Necessário atualizar as Secures IDs do bart

1. Visite https://bard.google.com/
2. Pressione F12 para console
3. Session: Application → Cookies → Copy the value of __Secure-1PSID and __Secure-1PSIDTS cookie.

In [ ]:

def bard_answer(texto):
#    token = 'Zwjn5-Mu593KxedqQOKifr-SLDW42MSVBvl2XnWj-r2idQeU_fn29np6lN_WKO6pcnCBbg.'
#    bard = Bard(token=token)


    cookie_dict = {
                    "__Secure-1PSID": "Zwjn5-Mu593KxedqQOKifr-SLDW42MSVBvl2XnWj-r2idQeU_fn29np6lN_WKO6pcnCBbg.",
                    "__Secure-1PSIDTS": "sidts-CjEBSAxbGZ9ChZ1Ug01LCTxeDyrzGF1fOgovjzvBfclhcZY60td2IgK-A2vv8yM46jotEAA",
                    }

    bard = BardCookies(cookie_dict=cookie_dict)
    resposta = bard.get_answer(texto)['content']
    return resposta

def load_Excel(tabela):
    resposta = []
    
    for i, info in tabela.iterrows():
        texto = f"Probabilidade do {info['Clube']} ser campeão brasileiro 2023 com {info['Pontos Ganhos']} pontos na rodada {info['Rodada Atual']}"
        resposta.append(bard_answer(texto))
 
    tabela['Probabilidade BART'] = resposta
    excel = pd.ExcelWriter('Desafio_DIO_IAGEN.xlsx', engine='xlsxwriter')
    tabela.to_excel(excel, sheet_name='Probabilidades GOOGLE BART')
    excel.save()



## EXECUÇÃO

In [ ]:
jn = get_equipes()
if  not (jn is None):
    tabela = transformacao(jn)
    load_Excel(tabela)